# Notebook 1 : Cupy + Xarray

Negin Sobhani and Deepak Cherian  
Computational & Information Systems Lab (CISL)  
negins@ucar.edu, dcherian@ucar.edu  

------------

## Introduction 

Xarray is a powerful library for working with labeled multi-dimensional arrays in Python. It provides a convenient and intuitive way to manipulate large and complex datasets, and is built on top of NumPy. CuPy, on the other hand, is a library that allows for efficient numerical computations on NVIDIA GPUs. 


When used together, Xarray and CuPy can provide an easy way to take advantage of GPU acceleration for scientific computing tasks. cupy-xarray provides an interface for using cupy in xarray, providing convenience accessors. 

In this tutorial, we'll explore how to use these two libraries together to perform high-performance computations on large datasets.

### Using CuPy with Xarray



In [7]:
## Import NumPy and CuPy
import cupy as cp
import numpy as np
import xarray as xr
import cupy_xarray # Adds .cupy to Xarray objects

#### Creating Xarray DataArray with CuPy

In [10]:
# create a DataArray with three dimensions and 100 elements along each dimension
da_np = xr.DataArray(np.random.rand(100, 100, 100), dims=['x', 'y', 'z'])

# create a DataArray with three dimensions and 100 elements along each dimension
da_cp = xr.DataArray(cp.random.rand(100, 100, 100), dims=['x', 'y', 'z'])

Check if these arrays are on GPU:

In [11]:
da_np.cupy.is_cupy

False

In [4]:
da_cp.cupy.is_cupy

True

In [5]:
da_cp.data.device

<CUDA Device 0>

### Performing Basic Operations with Xarray and CuPy

Once we have created a DataArray using CuPy, we can perform various operations on it using the familiar Xarray syntax. For example:

In [6]:
%%time
# calculate the mean along the x dimension
mean_da = da_cp.mean(dim='x')

# calculate the standard deviation along the y and z dimensions
std_da = da_cp.std(dim=['y', 'z'])

CPU times: user 874 ms, sys: 18.4 ms, total: 892 ms
Wall time: 968 ms


## Comparing Performance: CuPy with Xarray vs NumPy with Xarray
To compare the performance of using CuPy with Xarray to using NumPy with Xarray, let's perform a matrix multiplication operation using both libraries.

In [39]:
import time
# create two 1000x1000 DataArrays
n = 1000
da_np = xr.DataArray(np.random.rand(n, n), dims=['x', 'y'])
da_cp = xr.DataArray(cp.random.rand(n, n), dims=['x', 'y'])

# perform matrix multiplication with Xarray and NumPy
start_time = time.time()
result_np = da_np.dot(da_np)
end_time = time.time()
numpy_time = end_time - start_time

# perform matrix multiplication with Xarray and CuPy
start_time = time.time()
result_cp = da_cp.dot(da_cp)
cp.cuda.Stream.null.synchronize()  # wait for GPU computation to finish
end_time = time.time()
cupy_time = end_time - start_time

# calculate the speedup value with two decimal places
speedup = round(numpy_time / cupy_time, 2)

# print the speedup
print(f"CuPy with Xarray provides a {speedup:.2f}x speedup over NumPy with Xarray.")


CuPy with Xarray provides a 3.28x speedup over NumPy with Xarray.


Now, let's make the same comparison with other array sizes:

In [43]:
for n in [10, 100, 1000, 5000, 10000, 25000]:
    print("n =", n)

    da_np = xr.DataArray(np.random.rand(n, n), dims=['x', 'y'])
    da_cp = xr.DataArray(cp.random.rand(n, n), dims=['x', 'y'])

    # perform matrix multiplication with Xarray and NumPy
    start_time = time.time()
    result_np = da_np.dot(da_np)
    end_time = time.time()
    numpy_time = end_time - start_time

    # perform matrix multiplication with Xarray and CuPy
    start_time = time.time()
    result_cp = da_cp.dot(da_cp)
    cp.cuda.Stream.null.synchronize()  # wait for GPU computation to finish
    end_time = time.time()
    cupy_time = end_time - start_time

    print("Xarray DataArrays using CuPy provides a", round(numpy_time / cupy_time,2), "x speedup over NumPy.\n")

n = 10
Xarray DataArrays using CuPy provides a 1.0 x speedup over NumPy.

n = 100
Xarray DataArrays using CuPy provides a 0.77 x speedup over NumPy.

n = 1000
Xarray DataArrays using CuPy provides a 0.36 x speedup over NumPy.

n = 5000
Xarray DataArrays using CuPy provides a 26.18 x speedup over NumPy.

n = 10000
Xarray DataArrays using CuPy provides a 56.08 x speedup over NumPy.

n = 25000
Xarray DataArrays using CuPy provides a 85.47 x speedup over NumPy.



## High-level Xarray Functions: CuPy vs. NumPy

In this section of the tutorial, we'll explore the performance differences between high-level Xarray functions using CuPy and NumPy. CuPy is a GPU-based NumPy-compatible library, while NumPy is the well-known CPU-based library for numerical computations. We'll focus on three high-level functions: groupby, rolling mean, and weighted mean. We'll also compare the time it takes to execute each function using both CuPy and NumPy.
Let's create some sample data to work with.

We'll use a 3-dimensional dataset (time, latitude, longitude) with random values:

In [54]:
np.random.seed(0)
data_np = np.random.rand(1000, 180, 360)
data_cp = cp.array(data_np)
data_xr_np = xr.DataArray(data_np, dims=['time', 'lat', 'lon'])
data_xr_cp = xr.DataArray(data_cp, dims=['time', 'lat', 'lon'])

### Groupby
The groupby function is used to group data based on one or more dimensions. Here, we'll group our data by the 'time' dimension using both CuPy and NumPy:

In [55]:
start_time_np = time.time()

grouped_data_np = data_xr_np.groupby('time')

end_time_np = time.time()
time_np = end_time_np - start_time_np

In [1]:
start_time_cp = time.time()

grouped_data_cp = data_xr_cp.groupby('time')

end_time_cp = time.time()
time_cp = end_time_cp - start_time_cp

print("GroupBy with Xarray DataArrays using CuPy provides a", round(time_np / time_cp,2), "x speedup over NumPy.\n")

NameError: name 'time' is not defined

### Rolling Mean:

The rolling mean is a widely used technique for smoothing data over a specified window. We'll calculate the rolling mean along the 'time' dimension with a window size of 10:

In [58]:
start_time_np = time.time()

rolling_mean_np = data_xr_np.rolling(time=10).mean()

end_time_np = time.time()
time_np = end_time_np - start_time_np

In [60]:
start_time_cp = time.time()

rolling_mean_cp = data_xr_cp.rolling(time=10).mean()

end_time_cp = time.time()
time_cp = end_time_cp - start_time_cp

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

### Weighted Mean

The weighted mean is another way to smooth data, taking into account the varying importance of each data point. Here, we'll use a uniform weight along the 'time' dimension:



In [66]:
start_time_np = time.time()

weights_np = xr.DataArray(np.ones_like(data_np), dims="time")
weighted_mean_np = data_xr_np.weighted(weights_np).mean(dim='time')

end_time_np = time.time()
time_np = end_time_np - start_time_np


ValueError: different number of dimensions on data and dims: 3 vs 1

Plotting Works: